In [ ]:
import speech_recognition as sr
import pyttsx3
import time
import csv
from datetime import datetime

class SpeechDataEntrySystem:
    def __init__(self):
        # Initialize speech recognition
        self.recognizer = sr.Recognizer()
        self.microphone = sr.Microphone()
        
        # Initialize text-to-speech engine
        self.engine = pyttsx3.init()
        self.engine.setProperty('rate', 150)  # Speed of speech
        
        # Data storage
        self.data = []
        self.fields = ['Name', 'Age', 'Email', 'Phone', 'Timestamp']
        
        # Adjust for ambient noise
        with self.microphone as source:
            self.recognizer.adjust_for_ambient_noise(source)
            print("Adjusting for ambient noise...")
    
    def speak(self, text):
        """Convert text to speech and speak it"""
        print(f"System: {text}")
        self.engine.say(text)
        self.engine.runAndWait()
    
    def listen(self):
        """Listen to user speech and return recognized text"""
        with self.microphone as source:
            print("Listening...")
            audio = self.recognizer.listen(source)
        
        try:
            text = self.recognizer.recognize_google(audio)
            print(f"User: {text}")
            return text.lower()
        except sr.UnknownValueError:
            self.speak("Sorry, I didn't catch that. Could you please repeat?")
            return self.listen()
        except sr.RequestError:
            self.speak("Sorry, my speech service is down. Please try again later.")
            return None
    
    def collect_data(self):
        """Main method to collect data through voice"""
        self.speak("Welcome to the Speech Based Data Entry System.")
        
        while True:
            record = {}
            
            # Get name
            self.speak("Please say your full name.")
            name = self.listen()
            record['Name'] = name.title() if name else "Unknown"
            
            # Get age
            self.speak("Please say your age.")
            age = self.listen()
            record['Age'] = age if age else "Unknown"
            
            # Get email
            self.speak("Please say your email address.")
            email = self.listen()
            record['Email'] = email.replace(" at ", "@").replace(" dot ", ".") if email else "Unknown"
            
            # Get phone
            self.speak("Please say your phone number.")
            phone = self.listen()
            record['Phone'] = phone if phone else "Unknown"
            
            # Add timestamp
            record['Timestamp'] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            
            self.data.append(record)
            
            # Confirm
            self.speak(f"I have recorded: Name {record['Name']}, Age {record['Age']}, "
                      f"Email {record['Email']}, Phone {record['Phone']}. Is this correct?")
            confirmation = self.listen()
            
            if 'yes' in confirmation:
                self.speak("Record saved successfully.")
            else:
                self.data.pop()
                self.speak("Let's try that entry again.")
                continue
            
            # Ask to continue
            self.speak("Would you like to enter another record?")
            another = self.listen()
            if 'no' in another:
                break
        
        self.save_to_csv()
        self.speak("Thank you for using the Speech Based Data Entry System. Goodbye!")
    
    def save_to_csv(self):
        """Save collected data to a CSV file"""
        filename = f"speech_data_entry_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        with open(filename, 'w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=self.fields)
            writer.writeheader()
            writer.writerows(self.data)
        print(f"Data saved to {filename}")
    
    def run(self):
        """Run the data entry system"""
        try:
            self.collect_data()
        except KeyboardInterrupt:
            self.speak("Program terminated by user.")
            if self.data:
                self.save_to_csv()

if __name__ == "__main__":
    system = SpeechDataEntrySystem()
    system.run()